# xgboost_hyperparameter
## 結果
* 'max_depth': 20, 
* 'colsample_bytree': 0.6597946356014663, 
* 'subsample': 0.7, 
* 'gamma': 1.884172979513442, 
* 'lambda': 0.006050024963423825, 
* 'min_child_weight': 9, 
* 'num_boost_round': 926, 
* 'early_stopping_rounds': 98

best_value: 0.28495961062567476

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import optuna
import warnings
from tqdm import tqdm
warnings.simplefilter('ignore')

In [9]:
# 設定値
class CFG:
    # 変更するパラメータ
    n_folds = 5 # 公差検証の分割数
    n_trials = 100 # ハイパーパラメータチューニングの試行回数
    # その他設定値
    learning_rate = 0.01
    seed = 1234 

In [10]:
# BASE_DIR = '/kaggle/input/icr-identify-age-related-conditions/'
BASE_DIR = '../../data'
train_df = pd.read_csv(f'{BASE_DIR}/train.csv')
greeks_df = pd.read_csv(f'{BASE_DIR}/greeks.csv')
test_df = pd.read_csv(f'{BASE_DIR}/test.csv')
submission_df = pd.read_csv(f'{BASE_DIR}/sample_submission.csv')

In [11]:
train_df = train_df.drop('EJ', axis=1)
X_train = train_df.drop(['Id', 'Class'], axis=1)
y_train = train_df['Class']

### balanced loglossの計算（学習で使う？）

In [12]:
# 評価基準
def balanced_log_loss(y_true, y_pred):
    N = len(y_true)

    # Nc is the number of observations
    N_1 = np.sum(y_true == 1, axis=0)
    N_0 = np.sum(y_true == 0, axis=0)

    # In order to avoid the extremes of the log function, each predicted probability 𝑝 is replaced with max(min(𝑝,1−10−15),10−15)
    y_pred = np.maximum(np.minimum(y_pred, 1 - 1e-15), 1e-15)

    # balanced logarithmic loss
    loss_numerator = - (1/N_0) * np.sum((1 - y_true) * np.log(1-y_pred)) - (1/N_1) * np.sum(y_true * np.log(y_pred))

    return loss_numerator / 2

# Classの０，１の割合をそれぞれ計算
def calc_log_loss_weight(y_true):
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    return w0, w1

In [23]:
def objective(trial):
    # xgboost設定値
    xgb_params = {
        'objective': 'binary:logistic',# 学習タスク
        'tree_method': 'gpu_hist',
        'eval_metric': 'rmse',
        'random_state': CFG.seed,
        'learning_rate': CFG.learning_rate,
        # 探索するパラメータ
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 1.0]),
        'gamma': trial.suggest_uniform('gamma', 0, 2),
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
    }
    
    num_boost_round = trial.suggest_int('num_boost_round', 100, 1000)
    early_stopping_rounds = trial.suggest_int('early_stopping_rounds', 10, 100)
    
    scores = []
    # K-分割交差検証(層化抽出法)
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    
    for fold, (train_index, valid_index) in enumerate(kfold.split(X_train, y_train)):
        # 訓練データを分割
        X_train_fold = X_train.iloc[train_index]
        y_train_fold = y_train.iloc[train_index]
        X_valid_fold = X_train.iloc[valid_index]
        y_valid_fold = y_train.iloc[valid_index]
    
        # 訓練データの重みを計算
        train_w0, train_w1 = calc_log_loss_weight(y_train_fold)
        # 検証データの重みを計算
        valid_w0, valid_w1 = calc_log_loss_weight(y_valid_fold)
        # 訓練データをxgb用に変換
        xgb_train = xgb.DMatrix(data=X_train_fold, label=y_train_fold, weight=y_train_fold.map({0: train_w0, 1: train_w1}))
        # 検証データをxgb用に変換
        xgb_valid = xgb.DMatrix(data=X_valid_fold, label=y_valid_fold, weight=y_valid_fold.map({0: valid_w0, 1: valid_w1}))
        
        # モデルのインスタンス生成
        model = xgb.train(
            xgb_params, 
            dtrain = xgb_train, 
            num_boost_round = num_boost_round,
            evals = [(xgb_train, 'train'), (xgb_valid, 'eval')], 
            early_stopping_rounds = early_stopping_rounds,
            verbose_eval = False, # 整数に設定すると、n回ごとのブースティングステージで評価メトリクスを表示
        )
        # 予測
        preds = model.predict(xgb.DMatrix(X_valid_fold), iteration_range=(0, model.best_ntree_limit))
        # 予測値をラベルに変換
        # pred_labels = np.rint(preds)
        # 評価
        # val_score = balanced_log_loss(y_valid, pred_labels)
        val_score = balanced_log_loss(y_valid_fold, preds)
        
        scores.append(val_score)
    # クロスバリデーションの平均値を計算
    mean_score = np.mean(scores)
    
    return mean_score

In [24]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=CFG.n_trials)

[I 2023-07-24 17:55:19,509] A new study created in memory with name: no-name-8cb8a83a-1238-4d87-9349-aac3bb8ed438
[I 2023-07-24 17:55:25,108] Trial 0 finished with value: 0.4355671562350758 and parameters: {'max_depth': 1, 'colsample_bytree': 0.6423258532698192, 'subsample': 1.0, 'gamma': 0.9526436858780756, 'lambda': 0.2093470722704977, 'min_child_weight': 44, 'num_boost_round': 440, 'early_stopping_rounds': 69}. Best is trial 0 with value: 0.4355671562350758.


balanced_log_loss: 0.4355671562350758


[I 2023-07-24 17:55:35,542] Trial 1 finished with value: 0.30102973198448557 and parameters: {'max_depth': 7, 'colsample_bytree': 0.8117916986316722, 'subsample': 0.8, 'gamma': 0.25729557628452304, 'lambda': 0.0018937083495176763, 'min_child_weight': 19, 'num_boost_round': 773, 'early_stopping_rounds': 35}. Best is trial 1 with value: 0.30102973198448557.


balanced_log_loss: 0.30102973198448557


[I 2023-07-24 17:55:38,042] Trial 2 finished with value: 0.47588149270351254 and parameters: {'max_depth': 9, 'colsample_bytree': 0.8617425308624076, 'subsample': 0.6, 'gamma': 0.34976878977615034, 'lambda': 0.002233138842354782, 'min_child_weight': 42, 'num_boost_round': 164, 'early_stopping_rounds': 28}. Best is trial 1 with value: 0.30102973198448557.


...

balanced_log_loss: 0.2924643412068194


[I 2023-07-24 18:14:15,251] Trial 98 finished with value: 0.29062126450616804 and parameters: {'max_depth': 20, 'colsample_bytree': 0.641868832914191, 'subsample': 0.7, 'gamma': 1.8130809611288696, 'lambda': 0.012294009063102972, 'min_child_weight': 5, 'num_boost_round': 972, 'early_stopping_rounds': 70}. Best is trial 95 with value: 0.28495961062567476.


balanced_log_loss: 0.29062126450616804


[I 2023-07-24 18:14:30,785] Trial 99 finished with value: 0.29130930564741553 and parameters: {'max_depth': 20, 'colsample_bytree': 0.6598141164598006, 'subsample': 0.5, 'gamma': 1.8706914057028545, 'lambda': 0.007107374228627907, 'min_child_weight': 13, 'num_boost_round': 940, 'early_stopping_rounds': 95}. Best is trial 95 with value: 0.28495961062567476.


balanced_log_loss: 0.29130930564741553


In [25]:
best_params = study.best_trial.params
best_value = study.best_value
print("best_param: {}\n\nbest_value: {}".format(best_params, best_value))

best_param: {'max_depth': 20, 'colsample_bytree': 0.6597946356014663, 'subsample': 0.7, 'gamma': 1.884172979513442, 'lambda': 0.006050024963423825, 'min_child_weight': 9, 'num_boost_round': 926, 'early_stopping_rounds': 98}

best_value: 0.28495961062567476
